## Connecting graph-notebook to a Gremlin Server

![Gremlin](https://github.com/aws/graph-notebook/blob/main/images/gremlin-notebook.png?raw=true, "Picture of Gremlin holding a notebook")


See ["Connecting graph-notebook to a Gremlin Server"](https://github.com/aws/graph-notebook/tree/main/additional-databases/gremlin-server) for more details.
These notes explain how to connect the graph-notebook to a Gremlin server running locally on the same machine. The same steps should also work if you have a remote Gremlin Server. In such cases `localhost` should be replaced with the DNS or IP address of the remote server. It is assumed the [graph-notebook installation](https://github.com/aws/graph-notebook/blob/main/README.md) has been completed and the Jupyter environment is running before following these steps.


### Connecting to a local Gremlin Server from Jupyter
1. In the Jupyter Notebook disable SSL using `%%graph_notebook_config` and change the host to `localhost`. Keep the other defaults even though they are not used for configuring the Gremlin Server.
```
%%graph_notebook_config
{
    "host": "localhost",
    "port": 8182,
    "ssl": false,
    "proxy_port": 8192,
    "proxy_host": "",
    "auth_mode": "",
    "aws_region": "",
    "load_from_s3_arn": ""
}
```


### Connecting to a local Gremlin Server from Jupyter
- If the Gremlin Server you wish to connect to is remote,  replacing `localhost` with the IP address or DNS of the remote server should work. This assumes you have access to that server from your local machine.
- If you wish to connect to a Neptune Server from your local machine then you must have a viable Proxy solution to access the Neptune instance inside its VPC. This model can be seen in [this aws-samples post.](https://aws-samples.github.io/aws-dbs-refarch-graph/src/connecting-using-a-load-balancer/)(CDK Construct Pending).

#### Remote Gremlin Server
```
%%graph_notebook_config
{
    "host": "remote.gremlin.com",
    "port": 8182,
    "ssl": false,
    "proxy_port": 8192,
    "proxy_host": "",
    "auth_mode": "",
    "aws_region": "",
    "load_from_s3_arn": ""
}
```

#### Proxied Neptune Server
```
%%graph_notebook_config
{
    "host": "neptunedatabasecluster.cluster-XXXXXXX.us-east-1.neptune.amazonaws.com",
    "port": 8182,
    "ssl": true,
    "proxy_port": 8192,
    "proxy_host": "nlb.proxy.aws.com",
    "auth_mode": "IAM",
    "aws_region": "us-east-1",
    "load_from_s3_arn": ""
}
```
- IAM Authed connections through a proxied connection require the request to be signed with the HOST header set to the value of the Neptune endpoint, not the NLB endpoint. This process is handled by default in the [NeptuneUtils Library](https://github.com/awslabs/amazon-neptune-tools/tree/master/neptune-python-utils) published by neptune.
- Non-IAM Authed databases will not have to worry about this signing issue, however, it is recommended that there be some type of authentication before the request is allowed to interact with your database.


## Example NeptuneUtils Implementation

In [ ]:
from neptune_python_utils.gremlin_utils import GremlinUtils
from neptune_python_utils.endpoints import Endpoints

endpoints = Endpoints(
    neptune_endpoint='demo.cluster-111222333.eu-west-2.neptune.amazonaws.com', 
    region_name='us-east-1',
    proxy_dns='localhost',
    proxy_port=8182,
    remove_host_header=False)
    
GremlinUtils.init_statics(globals())
    
gremlin_utils = GremlinUtils(endpoints)

conn = None

try:
    conn = gremlin_utils.remote_connection(ssl=False)
    g = gremlin_utils.traversal_source(connection=conn)
    print(g.V().limit(10).valueMap().toList())
finally:
    if conn:
        conn.close()